# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter
from scipy.stats import percentileofscore

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
from tickers import get_tickers
from api_key import IEX_CLOUD_API_TOKEN
stocks = get_tickers()

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'aapl,zts'

api_url = f'https://api.iex.cloud/v1/data/core/quote,fundamental_valuations,fundamentals/{symbol}?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url)
response.status_code

200

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
data = response.json()
index = int(len(data)/3)
all_data = []
for i in range(0,index):
    data_dict = dict(
        ticker = data[i]['symbol'],
        price = data[i]['latestPrice'],
        per = data[i]['peRatio'],
        p_bv = data[i+index]['pToBv'],
        p_sales = data[i+index]['priceToRevenue'],
        ev_ebitda = data[i+index]['evToEbitda'],
        ev_sales = data[i+index]['evToSales'],
        ev = data[i+index]['enterpriseValue'],
        gross_profit = data[i+index*2]['profitGross'],
       )
    all_data.append(data_dict)

all_data

[{'ticker': 'AAPL',
  'price': 175.56,
  'per': 27.35,
  'p_bv': 40.65862310701754,
  'p_sales': 7.811141056218985,
  'ev_ebitda': 23.42454382271788,
  'ev_sales': 7.90172818734995,
  'ev': 3047743972230,
  'gross_profit': 54855000000},
 {'ticker': 'ZTS',
  'price': 153.645,
  'per': 30.3,
  'p_bv': 15.856779771563607,
  'p_sales': 9.619009399115996,
  'ev_ebitda': 26.356727735557616,
  'ev_sales': 10.191939753912315,
  'ev': 85316727680,
  'gross_profit': 5983000000}]

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 2000.

In [7]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunk (lst, n):
    for i in range(0,len(lst),n):
      yield lst[i:i+n]


def get_valuations_data(stock_list):
    symbol_groups= list(chunk(stock_list,2000))
    symbol_strings = []
    for i in range(0,len(symbol_groups)):
      symbol_strings.append(','.join(symbol_groups[i]))

    for symbol_string in symbol_strings:
        api_url = f'https://api.iex.cloud/v1/data/core/quote,fundamental_valuations/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()

        index = int(len(data)/2)
        all_data = []
        for i in range(0,index):
            data_dict = dict(
                ticker = data[i]['symbol'],
                price = data[i]['latestPrice'],
                per = data[i].get('peRatio',np.nan),
                p_bv = data[i+index].get('pToBv',np.nan) ,
                p_sales = data[i+index].get('priceToRevenue',np.nan),
                ev_ebitda = data[i+index].get('evToEbitda',np.nan),
                ev_sales = data[i+index].get('evToSales',np.nan),
                ev = data[i+index].get('enterpriseValue',np.nan),
                ev_gp = np.nan,
                shares_to_buy =np.nan)
            all_data.append(data_dict)

        df = pd.DataFrame(all_data)
        return df

It will be necessary to make a second API call to get the Gross Profit data for each company. The following cell contains the code to retrieve the symbols and the Gross Profit. We will join both data-frames to make our final data frame and take it from there.

In [8]:
def chunk (lst, n):
    for i in range(0,len(lst),n):
      yield lst[i:i+n]


def get_fundamentals_data(stock_list):
    symbol_groups= list(chunk(stock_list,2000))
    symbol_strings = []
    for i in range(0,len(symbol_groups)):
      symbol_strings.append(','.join(symbol_groups[i]))

    for symbol_string in symbol_strings:
            api_url = f'https://api.iex.cloud/v1/data/core/fundamentals/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
            data = requests.get(api_url).json()

            all_data = []
            for i in range(0,index):
                data_dict = dict(ticker = data[i]['symbol'],
                    gross_profit = data[i].get('profitGross',np.nan))

                all_data.append(data_dict)
            df = pd.DataFrame(all_data)
            return(df)



Now we need to merge both dataframes.

In [9]:
gp = get_fundamentals_data(stocks['Ticker'])
value = get_valuations_data(stocks['Ticker'])

value = pd.merge(value,gp,how='left',on='ticker')

In [10]:
value

,ticker,price,per,p_bv,p_sales,ev_ebitda,ev_sales,ev,ev_gp,shares_to_buy,gross_profit
0,MMM,91.600,-7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.420400e+10
1,AOS,86.550,23.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.484800e+09
2,ABT,109.985,33.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,11.085313,1.581537,-8.266207,1.913140,6.264958e+10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,1.849376,2.719831,18.793875,2.953244,1.994030e+10,NaN,NaN,NaN
495,GWW,963.330,26.39,-4.298150,4.986803,19.180202,6.526540,4.607084e+10,NaN,NaN,NaN
496,WYNN,100.655,15.56,4.042358,2.398508,16.504155,2.834112,1.439162e+10,NaN,NaN,NaN
497,XEL,53.210,16.58,1.876268,3.224708,14.956243,3.938719,2.867112e+10,NaN,NaN,NaN


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy.

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    while True:
        try:
            portfolio_size = float(portfolio_size)
            break
        except ValueError:
            print("That's not a number! \n Try again:")
            portfolio_size = input("Enter the value of your portfolio:")

## Building a Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
rv = value

# Calculate the EV/GP from the Enterprise value and Gross Profit:
rv['ev_gp'] = rv['ev']/rv['gross_profit']

#Create the new Columns:
rv[['per_percentile', 'p_bv_percentile', 'p_sales_percentile',	'ev_ebitda_percentile',	'ev_gp_percentile', 'rv_score']] = np.nan
rv = rv.drop(['ev','gross_profit'], axis=1)

rv

,ticker,price,per,p_bv,p_sales,ev_ebitda,ev_sales,ev_gp,shares_to_buy,per_percentile,p_bv_percentile,p_sales_percentile,ev_ebitda_percentile,ev_gp_percentile,rv_score
0,MMM,91.600,-7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,86.550,23.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABT,109.985,33.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,11.085313,1.581537,-8.266207,1.913140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,1.849376,2.719831,18.793875,2.953244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,GWW,963.330,26.39,-4.298150,4.986803,19.180202,6.526540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,WYNN,100.655,15.56,4.042358,2.398508,16.504155,2.834112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,XEL,53.210,16.58,1.876268,3.224708,14.956243,3.938719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
order_columns = ['ticker',	'price',	'per', 'per_percentile',	'p_bv', 'p_bv_percentile',
                 'p_sales','p_sales_percentile',	'ev_ebitda','ev_ebitda_percentile',	'ev_gp',
                 'ev_gp_percentile',	'shares_to_buy',	'rv_score']

rv= rv[order_columns]

In [14]:
rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,86.550,23.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABT,109.985,33.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,NaN,11.085313,NaN,1.581537,NaN,-8.266207,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,NaN,1.849376,NaN,2.719831,NaN,18.793875,NaN,NaN,NaN,NaN,NaN
495,GWW,963.330,26.39,NaN,-4.298150,NaN,4.986803,NaN,19.180202,NaN,NaN,NaN,NaN,NaN
496,WYNN,100.655,15.56,NaN,4.042358,NaN,2.398508,NaN,16.504155,NaN,NaN,NaN,NaN,NaN
497,XEL,53.210,16.58,NaN,1.876268,NaN,3.224708,NaN,14.956243,NaN,NaN,NaN,NaN,NaN


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' `info()` method to identify missing data:

In [15]:
rv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ticker                499 non-null    object 
 1   price                 499 non-null    float64
 2   per                   493 non-null    float64
 3   per_percentile        0 non-null      float64
 4   p_bv                  495 non-null    float64
 5   p_bv_percentile       0 non-null      float64
 6   p_sales               495 non-null    float64
 7   p_sales_percentile    0 non-null      float64
 8   ev_ebitda             495 non-null    float64
 9   ev_ebitda_percentile  0 non-null      float64
 10  ev_gp                 0 non-null      float64
 11  ev_gp_percentile      0 non-null      float64
 12  shares_to_buy         0 non-null      float64
 13  rv_score              0 non-null      float64
dtypes: float64(13), object(1)
memory usage: 54.7+ KB


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column.

Here is the code to do this:

In [16]:
for column in rv.columns[[2,4,6,8,10]]:
    rv.loc[:,column] = rv[column].fillna(rv[column].mean())

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.25,NaN,-7.101201,NaN,3.730284,NaN,20.451415,NaN,NaN,NaN,NaN,NaN
1,AOS,86.550,23.46,NaN,-7.101201,NaN,3.730284,NaN,20.451415,NaN,NaN,NaN,NaN,NaN
2,ABT,109.985,33.63,NaN,-7.101201,NaN,3.730284,NaN,20.451415,NaN,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,NaN,-7.101201,NaN,3.730284,NaN,20.451415,NaN,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,NaN,11.085313,NaN,1.581537,NaN,-8.266207,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,NaN,1.849376,NaN,2.719831,NaN,18.793875,NaN,NaN,NaN,NaN,NaN
495,GWW,963.330,26.39,NaN,-4.298150,NaN,4.986803,NaN,19.180202,NaN,NaN,NaN,NaN,NaN
496,WYNN,100.655,15.56,NaN,4.042358,NaN,2.398508,NaN,16.504155,NaN,NaN,NaN,NaN,NaN
497,XEL,53.210,16.58,NaN,1.876268,NaN,3.224708,NaN,14.956243,NaN,NaN,NaN,NaN,NaN


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
percentiles = rv.columns[[3,5,7,9,11]]
ratios = rv.columns[[2,4,6,8,10]]
metrics = dict(zip(ratios,percentiles))


for metric in metrics.keys():
  for row in rv.index:
    rv.loc[row,metrics[metric]] = stats.percentileofscore(rv[metric], rv.loc[row,metric])/100

rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.25,0.050100,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
1,AOS,86.550,23.46,0.504008,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
2,ABT,109.985,33.63,0.731463,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,0.915832,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,0.649299,11.085313,0.881764,1.581537,0.322645,-8.266207,0.014028,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,0.563126,1.849376,0.314629,2.719831,0.533066,18.793875,0.713427,NaN,NaN,NaN,NaN
495,GWW,963.330,26.39,0.567134,-4.298150,0.066132,4.986803,0.751503,19.180202,0.727455,NaN,NaN,NaN,NaN
496,WYNN,100.655,15.56,0.280561,4.042358,0.613226,2.398508,0.478958,16.504155,0.593186,NaN,NaN,NaN,NaN
497,XEL,53.210,16.58,0.320641,1.876268,0.326653,3.224708,0.595190,14.956243,0.496994,NaN,NaN,NaN,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv.index:
  rv.loc[row,'rv_score'] = mean(rv.loc[row,percentiles])

rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.25,0.050100,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
1,AOS,86.550,23.46,0.504008,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
2,ABT,109.985,33.63,0.731463,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.96,0.915832,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
4,ACN,316.930,28.76,0.649299,11.085313,0.881764,1.581537,0.322645,-8.266207,0.014028,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WTW,260.665,26.20,0.563126,1.849376,0.314629,2.719831,0.533066,18.793875,0.713427,NaN,NaN,NaN,NaN
495,GWW,963.330,26.39,0.567134,-4.298150,0.066132,4.986803,0.751503,19.180202,0.727455,NaN,NaN,NaN,NaN
496,WYNN,100.655,15.56,0.280561,4.042358,0.613226,2.398508,0.478958,16.504155,0.593186,NaN,NaN,NaN,NaN
497,XEL,53.210,16.58,0.320641,1.876268,0.326653,3.224708,0.595190,14.956243,0.496994,NaN,NaN,NaN,NaN


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv = rv.sort_values('rv_score')[0:50].reset_index(drop=True)
rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.250000,0.050100,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
1,AOS,86.550,23.460000,0.504008,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
2,ABT,109.985,33.630000,0.731463,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
3,ABBV,163.090,59.960000,0.915832,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,NaN,NaN
4,ACN,316.930,28.760000,0.649299,11.085313,0.881764,1.581537,0.322645,-8.266207,0.014028,NaN,NaN,NaN,NaN
5,ADBE,477.250,45.540000,0.851703,5.302211,0.703407,2.625995,0.513026,11.766662,0.324649,NaN,NaN,NaN,NaN
6,AMD,162.280,308.810000,0.997996,4.498736,0.649299,4.219754,0.693387,22.211090,0.811623,NaN,NaN,NaN,NaN
7,AES,16.525,47.250000,0.863727,21.803137,0.957916,4.782312,0.739479,12.377318,0.358717,NaN,NaN,NaN,NaN
8,AFL,80.720,10.380000,0.136273,8.775391,0.835671,3.685928,0.631263,19.094140,0.725451,NaN,NaN,NaN,NaN
9,A,140.700,33.580000,0.728457,16.507603,0.935872,12.801341,0.963928,35.302219,0.939880,NaN,NaN,NaN,NaN


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [22]:
position_size = float(portfolio_size/len(rv))

for row in rv.index:
  rv.loc[row,'shares_to_buy'] = math.floor(position_size/ rv.loc[row,'price'])

rv

,ticker,price,per,per_percentile,p_bv,p_bv_percentile,p_sales,p_sales_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,shares_to_buy,rv_score
0,MMM,91.600,-7.250000,0.050100,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,218.0,NaN
1,AOS,86.550,23.460000,0.504008,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,231.0,NaN
2,ABT,109.985,33.630000,0.731463,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,181.0,NaN
3,ABBV,163.090,59.960000,0.915832,-7.101201,0.055110,3.730284,0.646293,20.451415,0.766533,NaN,NaN,122.0,NaN
4,ACN,316.930,28.760000,0.649299,11.085313,0.881764,1.581537,0.322645,-8.266207,0.014028,NaN,NaN,63.0,NaN
5,ADBE,477.250,45.540000,0.851703,5.302211,0.703407,2.625995,0.513026,11.766662,0.324649,NaN,NaN,41.0,NaN
6,AMD,162.280,308.810000,0.997996,4.498736,0.649299,4.219754,0.693387,22.211090,0.811623,NaN,NaN,123.0,NaN
7,AES,16.525,47.250000,0.863727,21.803137,0.957916,4.782312,0.739479,12.377318,0.358717,NaN,NaN,1210.0,NaN
8,AFL,80.720,10.380000,0.136273,8.775391,0.835671,3.685928,0.631263,19.094140,0.725451,NaN,NaN,247.0,NaN
9,A,140.700,33.580000,0.728457,16.507603,0.935872,12.801341,0.963928,35.302219,0.939880,NaN,NaN,142.0,NaN


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('Value Strategy.xlsx', engine = 'xlsxwriter')
rv.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {'A': ['Ticker',string_template],
 'B': ['Price',dollar_template ],
 'C': ['P/E Ratio', float_template],
 'D': ['P/E Ratio Percentile', percent_template],
 'E': ['P/BV',float_template],
 'F': ['P/BV Percentile',percent_template],
 'G': ['P/Sales',float_template],
 'H': ['P/Sales Percentile',percent_template],
 'I': ['EV/EBITDA',float_template],
 'J': ['EV/EBITDA Percentile',percent_template],
 'K': ['EV/GP',float_template],
 'L': ['EV/GP Percentile',percent_template],
 'M': ['Shares to buy',integer_template],
 'N': ['RV Score',percent_template]

}

for column in column_formats.keys():
  writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
  writer.sheets['Value Strategy'].write(f'{column}1',column_formats[column][0],string_template)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.close()